## Purpose

Input data to the Raven model is constrained to the availability of future climate projections. For instance, the [Ontario Climate Data Portal (OCDP)](https://lamps.math.yorku.ca/OntarioClimate/) offers daily ensemble data from 1981 to 2099 as precipitation and min/max/mean air temperature. For the sake of preparing the Raven model to accept these data, input data during calibration has to be kept the same.

Raven offers a number of methodologies to partition precipitation into rainfall and snowfall on the basis of air temperature alone. Here, we are testing these methodologies on climate station data with reported rain and snow.

Tests will be performed at ECCC-AES climate station [GEORGETOWN WWTP (6152695)](https://climate.weather.gc.ca/climate_data/daily_data_e.html?StationID=4923) which has 30+ years of rain snow and temperature data

![](../img/6152695-annualMeanT.png)

*Annual average of daily mean air temperatures. Note that the completeness of data degrades sometime after 2005.*


## Raven methods

Raven offers four methods to determine the "snow fraction" $\alpha_s$ solely using precipitation and min/max/mean air temperatures. Since GEORGETOWN WWTP has both rainfall and snowfall reported along with min and max temperatures, the methods can be compared and chosen based on performance. *(See Raven model manual for more details.)*

$$ P = R + S $$

$$ R = (1-\alpha_s)P $$

$$ S = \alpha_s P $$

$$ T_\text{avg} = \frac{T_\text{max}+T_\text{min}}{2} $$

where $P$ is precipitation, $R$ is rainfall, $S$ is snowfall, and $T_\text{avg}$, $T_\text{max}$ and $T_\text{min}$ are the mean, max, min air temperatures, respectively.



### Temperature range approach (`RAINSNOW_DINGMAN`)

In the temperature range approach, the snow fraction, α, is calculated from the maximum and
minimum daily temperatures:

$$
  \alpha_s = \frac{T_\text{trans}-T_\text{min}}{T_\text{max}-T_\text{min}}
$$

If $T_\text{trans}$ is outside of this temperature range, the precipitation is either all snow $(\alpha_s=1)$ or all rain $(\alpha_s=0)$, accordingly.


### Temperature threshold approach (`RAINSNOW_THRESHOLD`)

In the temperature range approach, all precipitation is assumed to be snow when $T_\text{avg}<T_\text{trans}$, and
rain otherwise, where $T_\text{trans}$ is the rain/snow transition temperature.


### Linear approach (`RAINSNOW_HBV`)

In these approaches, a linear transition between all snow and all rain is determined from the average
daily temperature, $T_\text{avg}$:

$$
  \alpha_s = 0.5+\frac{T_\text{trans}-T_\text{avg}}{\Delta T}
$$

If $T_\text{trans}$ is outside of this temperature range, the precipitation is either all snow $(\alpha_s=1)$ or all rain $(\alpha_s=0)$, accordingly.


### The SNTHERM.89 model approach (`RAINSNOW_SNTHERM89`)

$$
  \alpha_s = 
  \begin{cases}
      0.0 & \text{if $T_\text{avg}>2.5$}\\
      0.6 & \text{if $T_\text{avg}>2.0$}\\
      1.0-0.2666\cdot (T_\text{avg}-0.5) & \text{if $T_\text{avg}>0.5$}\\
      1.0 & \text{otherwise}
    \end{cases}       
$$

## Testing

Using the R-script below, show the distibution precipitation type at GEORGETOWN WWTP.

```r
df <- read.csv('../dat/6152695.csv') %>%
  subset(select=-c(depth_of_surface_snow,mean_air_temperature,precipitation_amount)) %>%
  drop_na() %>%
  mutate(Tavg=(max_air_temperature+min_air_temperature)/2, precip=rainfall_amount+snowfall_amount) %>%
  filter(precip>0) %>%
  mutate(alpha=snowfall_amount/precip,stat=ifelse(alpha==1,'all rain', ifelse(alpha==0, 'all snow', 'mixed')))

df %>% ggplot() +
  theme_bw() + theme(legend.position = "inside", legend.position.inside = c(.1,.8), legend.title = element_blank()) +
  geom_density(aes(x=Tavg,y=after_stat(density * n/nrow(df)), fill=factor(stat)), alpha = 0.1, bw=1) +
  labs(title='distribuiton of precipitation type',subtitle='GEORGETOWN WWTP (6152695)',x='Air Temperature',y='density')
```

<img src="../dat/rain-snow-mixed-plot.png" width="600" />


### Optimization

The following code uses optimization routines to determine the best fitting parameter amongst the 4 Raven methods discussed above.

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit, basinhopping

# import data and prepare/clean
df = pd.read_csv("../dat/6152695.csv").drop(columns=['depth_of_surface_snow','mean_air_temperature','precipitation_amount'])
df['Tavg'] = (df.max_air_temperature+df.min_air_temperature)/2 # create mean temperature
df['precip'] = df.rainfall_amount+df.snowfall_amount # create precipitation
df['alpha'] = df.snowfall_amount/df.precip # determine actual snowfraction
df = df[df.max_air_temperature!=df.min_air_temperature] # remove dates with no temperature range (causes divides by zero below)
df.dropna(how='any', inplace=True) # get full data


# define functions based on equations above
## RAINSNOW_THRESHOLD_opt is a special form needed for the "basinhopping" algorithm (curve_fit will not work as RAINSNOW_THRESHOLD is a discontinuous function)
def RAINSNOW_THRESHOLD(Tavg, Tthr):
    a = np.zeros(len(Tavg))
    a[Tavg<Tthr] = 1
    return a

aa = np.array(df.alpha)
tt = np.array(df.Tavg)
def RAINSNOW_THRESHOLD_opt(Tthr):
    a = np.zeros(len(tt))
    a[tt<Tthr] = 1
    return np.sum((a-aa)**2)

def RAINSNOW_DINGMAN(Tminmax,Tthr):
    a = (Tthr-Tminmax[0])/(Tminmax[1]-Tminmax[0])
    a[a>1] = 1
    a[a<0] = 0
    return a

def RAINSNOW_HBV(Tavg,Tthr,delT):
    a = 0.5+(Tthr-Tavg)/delT
    a[a>1] = 1
    a[a<0] = 0
    return a #np.min(np.max(a,m0),m1)

def RAINSNOW_SNTHERM89(Tavg):
    a = 1.0-(Tavg-0.5)/3.75
    a[Tavg>2.0] = 0.6
    a[Tavg>2.5] = 0.0
    a[Tavg<=0.5] = 1.0
    return a


# solve (note RAINSNOW_SNTHERM89 has no parameters to solve)
res_thrs = basinhopping(RAINSNOW_THRESHOLD_opt, [0.], minimizer_kwargs={"method": "BFGS"},niter=10000)
res_ding, _ = curve_fit(RAINSNOW_DINGMAN, [np.array(df.min_air_temperature),np.array(df.max_air_temperature)], np.array(df.alpha), p0=(0.0))
res_hbv, _ = curve_fit(RAINSNOW_HBV, np.array(df.Tavg), np.array(df.alpha), p0=(0.0, 2.0))

# model with optimal parameters
df['RAINSNOW_THRESHOLD']=RAINSNOW_THRESHOLD(np.array(df.Tavg),res_thrs.x[0])
df['RAINSNOW_DINGMAN']=RAINSNOW_DINGMAN([np.array(df.min_air_temperature),np.array(df.max_air_temperature)],res_ding[0])
df['RAINSNOW_HBV']=RAINSNOW_HBV(np.array(df.Tavg),res_hbv[0],res_hbv[1])
df['RAINSNOW_SNTHERM89']=RAINSNOW_SNTHERM89(np.array(df.Tavg))

print(df)

# report findings, pick model with lowest RMSE
print("RAINSNOW_THRESHOLD; Tthr = {:.3f} -- RMSE = {:.3f}".format(res_thrs.x[0], np.sqrt(np.average((df.alpha-df.RAINSNOW_THRESHOLD)**2))))
print("RAINSNOW_DINGMAN; Tthr = {:.3f} -- RMSE = {:.3f}".format(res_ding[0], np.sqrt(np.average((df.alpha-df.RAINSNOW_DINGMAN)**2))))
print("RAINSNOW_HBV; Tthr = {:.3f}; delT = {:.3f} -- RMSE = {:.3f}".format(res_hbv[0],res_hbv[1], np.sqrt(np.average((df.alpha-df.RAINSNOW_HBV)**2))))
print("RAINSNOW_SNTHERM89 -- RMSE = {:.3f}".format(np.sqrt(np.average((df.alpha-df.RAINSNOW_SNTHERM89)**2))))


             Date  max_air_temperature  min_air_temperature  rainfall_amount  \
5699   1978-12-07                  3.5                 -2.0              0.4   
5700   1978-12-08                  2.5                 -1.0              0.0   
5712   1978-12-20                  1.5                 -9.5              5.4   
5718   1978-12-26                 -3.0                 -8.0              0.0   
5719   1978-12-27                 -3.0                 -6.5              0.0   
...           ...                  ...                  ...              ...   
19812  2024-08-17                 29.5                 22.5             24.5   
19822  2024-09-06                 24.0                 14.0              0.6   
19823  2024-09-09                 22.0                 10.5              6.0   
19831  2024-09-20                 26.0                 11.0              0.6   
19832  2024-09-23                 21.0                 14.0              3.0   

       snowfall_amount   Tavg  precip  

# Results

It appears that the linear approach (`RAINSNOW_HBV`) function performs best with the lowest RMSE. The final parameters are $T_\text{trans}=-1.63^{\circ}\text{C}$ and $\Delta T=11.722^{\circ}\text{C}$.

The respective Raven parameters are:

$$ \texttt{RAINSNOW\textunderscore TEMP}=-1.63^{\circ}\text{C} $$

$$ \texttt{RAINSNOW\textunderscore DELTA}=11.722^{\circ}\text{C} $$